In [2]:
!git clone https://github.com/microsoft/unilm.git
%cd unilm/speechlm

Cloning into 'unilm'...
remote: Enumerating objects: 9860, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 9860 (delta 59), reused 122 (delta 47), pack-reused 9711
Receiving objects: 100% (9860/9860), 58.69 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (4610/4610), done.
Updating files: 100% (6571/6571), done.
/kaggle/working/unilm/speechlm


In [7]:
!pip install gdown
!gdown --fuzzy "https://drive.google.com/file/d/1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi/view?usp=sharing"

Downloading...
From (original): https://drive.google.com/uc?id=1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi
From (redirected): https://drive.google.com/uc?id=1azAQ_0AiwYSbmpwyTECCcraYKMkHBCxi&confirm=t&uuid=bf34a8cf-8250-4149-8427-b6cfaa0dd756
To: /kaggle/working/unilm/speechlm/speechlmh_base_checkpoint_clean.pt
100%|████████████████████████████████████████| 389M/389M [00:08<00:00, 43.7MB/s]


In [9]:
import torch
import torch.nn.functional as F
from SpeechLM import SpeechLMConfig, SpeechLM

checkpoint = torch.load('speechlmh_base_checkpoint_clean.pt')
cfg = SpeechLMConfig(checkpoint['cfg']['model'])
model = SpeechLM(cfg)
model.load_state_dict(checkpoint['model'])
model.eval()

wav_input_16khz = torch.randn(1,10000)
normalize = checkpoint['cfg']['task']['normalize']  # False for base model, True for large model
if normalize:
    wav_input_16khz = F.layer_norm(wav_input_16khz[0], wav_input_16khz[0].shape).unsqueeze(0)

# extract the representation of last layer
rep = model.extract_features(wav_input_16khz)[0]

# # extract the representation of each layer
# output_layer = model.cfg.encoder_layers + model.cfg.text_transformer.encoder.layers
# rep, layer_results = model.extract_features(wav_input_16khz, output_layer=output_layer, ret_layer_results=True)[0]
# layer_reps = [x.transpose(0, 1) for x in layer_results]

In [11]:
rep.shape

torch.Size([1, 31, 768])

In [ ]:
#TBD